## Deliverable 2. Create a Customer Travel Destinations Map.

In [17]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [18]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Atuona,PF,-9.8000,-139.0333,79.77,75,20,18.86,few clouds
1,1,Pitea,SE,65.3172,21.4794,23.09,85,99,3.00,overcast clouds
2,2,Busselton,AU,-33.6500,115.3333,69.33,83,91,18.90,overcast clouds
3,3,Biak,ID,-0.9131,122.8766,82.67,73,98,2.77,overcast clouds
4,4,Birobidzhan,RU,48.8000,132.9500,45.39,71,29,8.63,scattered clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 95


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Atuona,PF,-9.8000,-139.0333,79.77,75,20,18.86,few clouds
3,3,Biak,ID,-0.9131,122.8766,82.67,73,98,2.77,overcast clouds
8,8,Kapaa,US,22.0752,-159.3190,78.78,92,75,26.46,heavy intensity rain
10,10,Airai,TL,-8.9266,125.4092,75.33,48,32,2.19,scattered clouds
11,11,Hilo,US,19.7297,-155.0900,73.13,88,75,5.75,moderate rain
17,17,Gamba,GA,-2.6500,10.0000,74.52,95,91,4.43,overcast clouds
20,20,Avarua,CK,-21.2078,-159.7750,80.65,83,78,2.30,broken clouds
21,21,Butaritari,KI,3.0707,172.7902,82.04,77,23,15.95,light rain
22,22,Altamira,BR,-3.2033,-52.2064,72.25,99,96,1.70,overcast clouds
26,26,Hithadhoo,MV,-0.6000,73.0833,83.61,68,11,5.35,few clouds


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                247
City                   247
Country                247
Lat                    247
Lng                    247
Max Temp               247
Humidity               247
Cloudiness             247
Wind Speed             247
Current Description    247
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# there are no empty rows
clean_df = preferred_cities_df
clean_df.count()

City_ID                247
City                   247
Country                247
Lat                    247
Lng                    247
Max Temp               247
Humidity               247
Cloudiness             247
Wind Speed             247
Current Description    247
dtype: int64

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Atuona,PF,79.77,few clouds,-9.8000,-139.0333,
3,Biak,ID,82.67,overcast clouds,-0.9131,122.8766,
8,Kapaa,US,78.78,heavy intensity rain,22.0752,-159.3190,
10,Airai,TL,75.33,scattered clouds,-8.9266,125.4092,
11,Hilo,US,73.13,moderate rain,19.7297,-155.0900,
17,Gamba,GA,74.52,overcast clouds,-2.6500,10.0000,
20,Avarua,CK,80.65,broken clouds,-21.2078,-159.7750,
21,Butaritari,KI,82.04,light rain,3.0707,172.7902,
22,Altamira,BR,72.25,overcast clouds,-3.2033,-52.2064,
26,Hithadhoo,MV,83.61,few clouds,-0.6000,73.0833,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Get the name of the City
    city = row["City"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# 7. Check rows to see if count matches
hotel_df.count()

City                   247
Country                247
Max Temp               247
Current Description    247
Lat                    247
Lng                    247
Hotel Name             247
dtype: int64

In [12]:
#7a. Drop any rows where there is no hotel name
clean_hotel_df = hotel_df.dropna(how='any')
clean_hotel_df.count()

City                   247
Country                247
Max Temp               247
Current Description    247
Lat                    247
Lng                    247
Hotel Name             247
dtype: int64

In [13]:
clean_hotel_df.count()

City                   247
Country                247
Max Temp               247
Current Description    247
Lat                    247
Lng                    247
Hotel Name             247
dtype: int64

In [14]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)


# 11b. Display the figure
#fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))